### A Questionable Neural Network Approach

In [1]:
import pathlib
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from transformers import AdamW, get_linear_schedule_with_warmup

# get the absolute path of the notebook's directory
notebook_dir = pathlib.Path().resolve()

# append the notebook's directory to the Python interpreter's search path
sys.path.append(str(notebook_dir.parent))

class HealthDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# neural network model with self-attention and LSTM
class HealthRiskModel(nn.Module):
    def __init__(self, input_dim):
        super(HealthRiskModel, self).__init__()
        self.embedding = nn.Linear(input_dim, 128)
        self.lstm = nn.LSTM(128, 64, num_layers=2, batch_first=True, bidirectional=True)
        self.attention = nn.MultiheadAttention(128, num_heads=4)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x.unsqueeze(1))
        x = x.squeeze(1)
        x = x.unsqueeze(1)
        x, _ = self.attention(x, x, x)
        x = x.squeeze(1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

data = pd.read_csv('../data/1m_health_events_dataset.csv')

# convert 'Severity' column to numeric values
severity_mapping = {'low': 0, 'medium': 1, 'high': 2}
data['Severity'] = data['Severity'].map(severity_mapping).astype(int)

# perform one-hot encoding for categorical columns
data = pd.get_dummies(data, columns=['EventType', 'Location'])

# drop unnecessary columns
data.drop(columns=['Timestamp', 'Details'], inplace=True)

# convert DataFrame to numpy array
X = data.drop(columns=['Is_Anomaly']).values.astype(np.float32)
y = data['Is_Anomaly'].values.astype(np.float32)

# scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = HealthDataset(X_train, y_train)
test_dataset = HealthDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

input_dim = X_train.shape[1]
model = HealthRiskModel(input_dim)

criterion = nn.BCELoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

num_epochs = 10
num_training_steps = num_epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# now we are ready to train the model!
best_f1 = 0.0
for epoch in range(num_epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

    model.eval()
    with torch.no_grad():
        y_pred = []
        y_true = []
        for features, labels in test_loader:
            outputs = model(features)
            y_pred.extend(outputs.round().squeeze().tolist())
            y_true.extend(labels.tolist())

        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print()

        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), "nn_best_model.pt")

best_model = HealthRiskModel(input_dim)
best_model.load_state_dict(torch.load("nn_best_model.pt"))
best_model.eval()

/Users/xfu/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/Users/xfu/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/10
Accuracy: 0.9998
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000

Epoch 2/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.0816
F1-score: 0.1509

Epoch 3/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934

Epoch 4/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934

Epoch 5/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934

Epoch 6/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934

Epoch 7/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934

Epoch 8/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934

Epoch 9/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934

Epoch 10/10
Accuracy: 0.9998
Precision: 1.0000
Recall: 0.2449
F1-score: 0.3934



HealthRiskModel(
  (embedding): Linear(in_features=15, out_features=128, bias=True)
  (lstm): LSTM(128, 64, num_layers=2, batch_first=True, bidirectional=True)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
  )
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)